In [1]:
import numpy as np
import tensorflow as tf
import random as rn

seed = 42
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)

In [2]:
import os
import numpy as np 
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.graph_objs as go

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV, cross_validate, GroupKFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import *

import xgboost as xgb 
import lightgbm as lgb
import bayes_opt
import itertools

from bayes_opt import BayesianOptimization

# Deep learning packages
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Dropout, LSTM
import matplotlib.pyplot as plt

# import pycaret

# 폰트 설정 - 윈도우
plt.rcParams['font.family'] = 'NanumGothic'

# 그래프의 크기
plt.rcParams['figure.figsize'] = 12, 6

# 글자 크기
plt.rcParams['font.size'] = 14

# 폰트 설정 시 - 기호 깨는거 방지하기
plt.rcParams['axes.unicode_minus'] = False

# 불필요한 경고 메시지를 나오지 않도록 한다.
import warnings
warnings.filterwarnings('ignore')

In [3]:
from fbprophet import Prophet

In [4]:
def nmae(true_df, pred_df):
#     target_idx = true_df.iloc[:,0]
#     pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
#     pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
#     true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
#     true = true_df.iloc[:,1].to_numpy()
#     pred = pred_df.iloc[:,1].to_numpy()
    true = true_df.iloc[:].to_numpy()
    pred = pred_df.iloc[:].to_numpy()
    
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

def nmae_values(true, pred):
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [10]:
seoul = pd.read_csv('./data/weather/seoul_test.csv', encoding = 'euc-kr')
seoul = seoul[['일시', '기온(°C)', '풍속(m/s)', '습도(%)', '지면온도(°C)']]
seoul.columns = ['일시', '기온', '풍속', '습도', '지면온도']
def weather_interpolate(weather_data) :
  df = weather_data.copy()
  col_idx = weather_data[['기온', '풍속', '습도', '지면온도']].isna().sum().index
  for col in col_idx :
    print('{}의 결측값 대체중'.format(col))
    df[[col]] = df[[col]].interpolate()
  print('완료')
  return df

weather_ = weather_interpolate(seoul)
weather_train = weather_[ : -2160]
weather_test = weather_[-2160 : ].reset_index(drop = True)

기온의 결측값 대체중
풍속의 결측값 대체중
습도의 결측값 대체중
지면온도의 결측값 대체중
완료


In [11]:
weather_train

,일시,기온,풍속,습도,지면온도
0,2013-01-01 01:00:00,-8.5,4.0,57.0,-3.4
1,2013-01-01 02:00:00,-8.4,4.0,60.0,-3.4
2,2013-01-01 03:00:00,-8.1,3.4,58.0,-3.4
3,2013-01-01 04:00:00,-8.2,3.8,58.0,-3.4
4,2013-01-01 05:00:00,-8.2,4.4,61.0,-3.3
...,...,...,...,...,...
52579,2018-12-31 20:00:00,-3.7,0.8,39.0,-3.0
52580,2018-12-31 21:00:00,-4.6,1.1,44.0,-4.1
52581,2018-12-31 22:00:00,-5.4,1.3,46.0,-5.0
52582,2018-12-31 23:00:00,-5.2,1.6,47.0,-5.1


In [14]:
series = weather_train[['일시', '기온']]
# idx = df[df[col].isna()].index.tolist()

series.columns = ['ds', 'y']

# 모델 생성
model = Prophet()
# 모델 학습
model.fit(series)

In [16]:
series['y'].min()

-18.0

In [17]:
from tqdm import tqdm
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot

from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
# 최적의 매개변수 찾기 - 교차 검증 및 rmse 

# 예측값 상한,하한 제어 
series['cap'] = 40
series['floor'] = -20

best_score = 5

for changepoint_prior_scale in tqdm([0.001, 0.0025, 0.005, 0.008]):
    for seasonality_prior_scale in [0.01, 0.015, 0.02, 0.25]:
        for holidays_prior_scale in [0.01, 0.05, 0.1, 0.5, 0.8]:
            # 매개변수 각 조합에 대해 frophet 훈련
            m = Prophet(changepoint_prior_scale=changepoint_prior_scale, 
                        growth='logistic',      # 상한, 하한 제한
                        seasonality_mode='multiplicative',
                        yearly_seasonality=True,  # True
                        weekly_seasonality=20, 
                        daily_seasonality=False,  # false
                        seasonality_prior_scale = seasonality_prior_scale,
                        holidays_prior_scale=holidays_prior_scale, 
                       )
            m.fit(series)
            # 교차 검증 적용
            df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')
            df_p = performance_metrics(df_cv)
            # 교차 검증 rmse 평균을 계산
            score = df_p['rmse'].mean()
            # rmse가 best_score보다 낮으면, 매개변수와 함께 기록
            if score < best_score:
                best_score = score
                best_parameters = {'changepoint_prior_scale':changepoint_prior_scale, 'seasonality_prior_scale':seasonality_prior_scale,
                                  'holidays_prior_scale':holidays_prior_scale}

print('best rmse score(in cross_val): ', best_score)
print('최적의 매개변수: ', best_parameters)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]INFO:fbprophet:Making 7 forecasts with cutoffs between 2015-01-17 00:00:00 and 2018-01-01 00:00:00


  0%|          | 0/7 [00:00<?, ?it/s]

INFO:fbprophet:Skipping MAPE because y close to 0
INFO:fbprophet:Making 7 forecasts with cutoffs between 2015-01-17 00:00:00 and 2018-01-01 00:00:00


  0%|          | 0/7 [00:00<?, ?it/s]

INFO:fbprophet:Skipping MAPE because y close to 0
INFO:fbprophet:Making 7 forecasts with cutoffs between 2015-01-17 00:00:00 and 2018-01-01 00:00:00


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|                                                                                            | 0/4 [13:55<?, ?it/s]


KeyboardInterrupt: 